In [102]:
import json
import pandas as pd

In [103]:
nodeData = open("../06-PedestrianSimulation/ped_sim/data/NetworkMap.json").read()

In [104]:
data = json.loads(nodeData)

In [105]:
records = []
for record in data:
    nodeId = data[str(record)][0]['NodeID']
    nodeVolume = data[str(record)][0]['NodeVolume']
    nodeFeederCount = data[str(record)][0]['NodeFeederCount']
    borders = data[str(record)][0]['Borders']
    levelOneConnections = data[str(record)][0]['LevelOneConnections']
    object = {'nodeID':nodeId,'nodeVolume':nodeVolume,'nodeFeederCount':nodeFeederCount,'borders':borders,'levelOneConnections':levelOneConnections}
    records.append(object)
    
    

In [106]:
df_raw = pd.DataFrame(records)

In [107]:
df = df_raw.copy()
df['nodeID'] = df_raw['nodeID'] - 1
for i in range(len(df)):
    connections = df.iloc[i]['levelOneConnections']
    connections = [x-1 for x in connections]
    df.at[i,'levelOneConnections'] = connections
df.head()

,borders,levelOneConnections,nodeFeederCount,nodeID,nodeVolume
0,"[{'WaterBorder': 0.5, 'GovernmentBorder': 0.25...","[1, 3, 20]",3,0,328
1,"[{'WaterBorder': 0.5, 'GovernmentBorder': 0.25...","[0, 5]",4,1,2090
2,"[{'WaterBorder': 0.0, 'GovernmentBorder': 0.0,...","[3, 19]",4,2,1181
3,"[{'WaterBorder': 0.0, 'GovernmentBorder': 0.25...","[0, 2, 4, 5]",4,3,1892
4,"[{'WaterBorder': 0.0, 'GovernmentBorder': 0.0,...","[3, 6, 19]",4,4,1562


In [116]:
df['levelTwoConnections'] = ""
df['levelThreeConnections'] = ""


for i in range(len(df)):
    two_connections = []
    one_connections = df.iloc[i]['levelOneConnections']
    for connection in one_connections:
        a = df.iloc[connection]['levelOneConnections']
        for item in a:
            two_connections.append(item)
    df.at[i, 'levelTwoConnections'] = list(set(two_connections))
    
for i in range(len(df)):
    three_connections = []
    two_connections = df.iloc[i]['levelTwoConnections']
    for connection in two_connections:
        a = df.iloc[connection]['levelTwoConnections']
        for item in a:
            three_connections.append(item)
    df.at[i, 'levelThreeConnections'] = list(set(three_connections))
    
    

In [117]:
def countVolume(data,nodes):
    v = 0
    for i in nodes:
        a = data.loc[i]['nodeVolume']
        v += a
        
    return v

In [118]:
countVolume(df,[0,5])

5546

In [126]:
L1_volume = []
L2_volume = []
L3_volume = []

for i in range(len(df)):
    L1_nodes = df.iloc[i]['levelOneConnections']
    L2_nodes = df.iloc[i]['levelTwoConnections']
    L3_nodes = df.iloc[i]['levelThreeConnections']
    
    L1_v = countVolume(df, L1_nodes)
    L2_v = countVolume(df, L2_nodes)
    L3_v = countVolume(df, L3_nodes)
    
    L1_volume.append(L1_v)
    L2_volume.append(L2_v)
    L3_volume.append(L3_v)
    
df['L1_Volume'] = L1_volume
df['L2_Volume'] = L2_volume
df['L3_Volume'] = L3_volume

df.head()

,borders,levelOneConnections,nodeFeederCount,nodeID,nodeVolume,levelTwoConnections,levelThreeConnections,L1_Volume,L2_Volume,L3_Volume
0,"[{'WaterBorder': 0.5, 'GovernmentBorder': 0.25...","[1, 3, 20]",3,0,328,"[0, 2, 4, 5]","[0, 2, 4, 5, 8, 9]",4683,8289,10762
1,"[{'WaterBorder': 0.5, 'GovernmentBorder': 0.25...","[0, 5]",4,1,2090,"[1, 3, 6, 7, 20]","[1, 3, 6, 7, 10, 12, 13, 19, 20]",5546,10116,11710
2,"[{'WaterBorder': 0.0, 'GovernmentBorder': 0.0,...","[3, 19]",4,2,1181,"[0, 2, 4, 5]","[0, 2, 4, 5, 8, 9]",2698,8289,10762
3,"[{'WaterBorder': 0.0, 'GovernmentBorder': 0.25...","[0, 2, 4, 5]",4,3,1892,"[1, 3, 6, 7, 19, 20]","[1, 3, 6, 7, 10, 12, 13, 19, 20]",8289,10922,11710
4,"[{'WaterBorder': 0.0, 'GovernmentBorder': 0.0,...","[3, 6, 19]",4,4,1562,"[0, 2, 4, 5, 8]","[0, 2, 4, 5, 8, 9, 11, 12, 13, 14, 15]",5388,10041,11183
